# Arduino Debugging & Testing Notebook

Notebook ini digunakan untuk debugging dan testing sistem Arduino:
- Mengecek instalasi Arduino CLI
- Testing kompilasi dan upload kode Arduino
- Testing komunikasi serial
- Debugging sensor IR dan motor
- Validasi sistem keseluruhan

In [1]:
import subprocess
import os
import time
import serial
import sys
from datetime import datetime

In [2]:
# Konfigurasi
ARDUINO_FILE = 'plasticbottle_rvm.ino'
ARDUINO_PORT = '/dev/ttyACM0'
ARDUINO_BOARD = 'arduino:avr:mega'

def check_arduino_cli():
    """Cek instalasi dan versi Arduino CLI"""
    print("🔍 Checking Arduino CLI installation...")
    
    try:
        result = subprocess.run(['arduino-cli', 'version'], 
                              capture_output=True, text=True, timeout=10)
        
        if result.returncode == 0:
            print(f"✅ Arduino CLI found: {result.stdout.strip()}")
            return True
        else:
            print(f"❌ Arduino CLI error: {result.stderr}")
            return False
            
    except FileNotFoundError:
        print("❌ Arduino CLI not found!")
        print("💡 Install with: curl -fsSL https://raw.githubusercontent.com/arduino/arduino-cli/master/install.sh | sh")
        return False
    except Exception as e:
        print(f"❌ Error checking Arduino CLI: {e}")
        return False

def check_arduino_core():
    """Cek instalasi Arduino core"""
    print("\n🔍 Checking Arduino cores...")
    
    try:
        result = subprocess.run(['arduino-cli', 'core', 'list'], 
                              capture_output=True, text=True, timeout=10)
        
        print(f"📋 Installed cores:\n{result.stdout}")
        
        if 'arduino:avr' in result.stdout:
            print("✅ Arduino AVR core is installed")
            return True
        else:
            print("❌ Arduino AVR core not found!")
            print("💡 Install with: arduino-cli core install arduino:avr")
            return False
            
    except Exception as e:
        print(f"❌ Error checking cores: {e}")
        return False

def check_arduino_file():
    """Cek file Arduino"""
    print(f"\n🔍 Checking Arduino file: {ARDUINO_FILE}")
    
    if os.path.exists(ARDUINO_FILE):
        with open(ARDUINO_FILE, 'r') as f:
            content = f.read()
            
        print(f"✅ File found: {len(content)} characters")
        print(f"📋 First 5 lines:")
        
        lines = content.split('\n')[:5]
        for i, line in enumerate(lines, 1):
            print(f"  {i}: {line}")
            
        return True
    else:
        print(f"❌ File {ARDUINO_FILE} not found!")
        return False

def check_arduino_port():
    """Cek port Arduino"""
    print(f"\n🔍 Checking Arduino port: {ARDUINO_PORT}")
    
    # List all available ports
    try:
        ports = []
        for port in ['/dev/ttyACM0', '/dev/ttyACM1', '/dev/ttyUSB0', '/dev/ttyUSB1']:
            if os.path.exists(port):
                ports.append(port)
        
        print(f"📋 Available ports: {ports}")
        
        if ARDUINO_PORT in ports:
            print(f"✅ Port {ARDUINO_PORT} is available")
            return True
        elif ports:
            print(f"⚠️ Port {ARDUINO_PORT} not found, but alternatives available: {ports}")
            return True
        else:
            print("❌ No Arduino ports found!")
            return False
            
    except Exception as e:
        print(f"❌ Error checking ports: {e}")
        return False

# Jalankan semua pengecekan
print("🚀 Arduino System Diagnostic")
print("=" * 50)

cli_ok = check_arduino_cli()
core_ok = check_arduino_core()
file_ok = check_arduino_file()
port_ok = check_arduino_port()

print("\n" + "=" * 50)
print("📊 DIAGNOSTIC SUMMARY:")
print(f"Arduino CLI: {'✅' if cli_ok else '❌'}")
print(f"Arduino Core: {'✅' if core_ok else '❌'}")
print(f"Arduino File: {'✅' if file_ok else '❌'}")
print(f"Arduino Port: {'✅' if port_ok else '❌'}")

all_ok = cli_ok and core_ok and file_ok and port_ok
print(f"\n🎯 System Ready: {'✅ YES' if all_ok else '❌ NO'}")

🚀 Arduino System Diagnostic
🔍 Checking Arduino CLI installation...
✅ Arduino CLI found: arduino-cli  Version: 1.1.1-arch Commit: fa6eafcbbea301eeece900f0501e88d288487974 Date:

🔍 Checking Arduino cores...
📋 Installed cores:
ID              Installed Latest Name
arduino:avr     1.8.6     1.8.6  Arduino AVR Boards
esp32:esp32     3.2.0     3.2.0  esp32
esp8266:esp8266 3.1.2     3.1.2  esp8266


✅ Arduino AVR core is installed

🔍 Checking Arduino file: plasticbottle_rvm.ino
✅ File found: 1787 characters
📋 First 5 lines:
  1: // Pin infrared
  2: const int irPins[] = {A0, A1, A2, A3};
  3: 
  4: // Motor DC pin
  5: const int ENA = 8;

🔍 Checking Arduino port: /dev/ttyACM0
📋 Available ports: ['/dev/ttyACM0']
✅ Port /dev/ttyACM0 is available

📊 DIAGNOSTIC SUMMARY:
Arduino CLI: ✅
Arduino Core: ✅
Arduino File: ✅
Arduino Port: ✅

🎯 System Ready: ✅ YES


In [3]:
def test_arduino_compile():
    """Test kompilasi Arduino tanpa upload"""
    print("🔨 Testing Arduino compilation...")
    print(f"⏰ Started at: {datetime.now().strftime('%H:%M:%S')}")
    
    try:
        # Compile only (tanpa upload)
        cmd = [
            'arduino-cli', 'compile',
            '--fqbn', ARDUINO_BOARD,
            '--verbose',
            ARDUINO_FILE
        ]
        
        print(f"🖥️ Running: {' '.join(cmd)}")
        
        result = subprocess.run(cmd, capture_output=True, text=True, timeout=120)
        
        print(f"\n📤 Return code: {result.returncode}")
        
        if result.stdout:
            print(f"\n📤 STDOUT (last 10 lines):")
            stdout_lines = result.stdout.split('\n')[-10:]
            for line in stdout_lines:
                if line.strip():
                    print(f"  {line}")
        
        if result.stderr:
            print(f"\n📥 STDERR:")
            stderr_lines = result.stderr.split('\n')
            for line in stderr_lines:
                if line.strip():
                    print(f"  {line}")
        
        if result.returncode == 0:
            print("\n✅ Compilation successful!")
            return True
        else:
            print("\n❌ Compilation failed!")
            return False
            
    except subprocess.TimeoutExpired:
        print("\n❌ Compilation timeout (>120s)")
        return False
    except Exception as e:
        print(f"\n❌ Error during compilation: {e}")
        return False

def test_arduino_upload():
    """Test upload Arduino ke board"""
    print("\n📤 Testing Arduino upload...")
    print(f"⏰ Started at: {datetime.now().strftime('%H:%M:%S')}")
    
    try:
        cmd = [
            'arduino-cli', 'compile', '--upload',
            '--fqbn', ARDUINO_BOARD,
            '--port', ARDUINO_PORT,
            '--verbose',
            ARDUINO_FILE
        ]
        
        print(f"🖥️ Running: {' '.join(cmd)}")
        
        result = subprocess.run(cmd, capture_output=True, text=True, timeout=180)
        
        print(f"\n📤 Return code: {result.returncode}")
        
        if result.stdout:
            print(f"\n📤 STDOUT (last 10 lines):")
            stdout_lines = result.stdout.split('\n')[-10:]
            for line in stdout_lines:
                if line.strip():
                    print(f"  {line}")
        
        if result.stderr:
            print(f"\n📥 STDERR:")
            stderr_lines = result.stderr.split('\n')
            for line in stderr_lines:
                if line.strip():
                    print(f"  {line}")
        
        if result.returncode == 0:
            print("\n✅ Upload successful!")
            print("⏱️ Waiting for Arduino restart...")
            time.sleep(3)
            return True
        else:
            print("\n❌ Upload failed!")
            return False
            
    except subprocess.TimeoutExpired:
        print("\n❌ Upload timeout (>180s)")
        return False
    except Exception as e:
        print(f"\n❌ Error during upload: {e}")
        return False

# Test kompilasi
print("\n🧪 ARDUINO COMPILATION & UPLOAD TEST")
print("=" * 50)

compile_ok = test_arduino_compile()

if compile_ok:
    upload_ok = test_arduino_upload()
else:
    print("⚠️ Skipping upload test due to compilation failure")
    upload_ok = False

print("\n" + "=" * 50)
print("📊 TEST RESULTS:")
print(f"Compilation: {'✅ PASS' if compile_ok else '❌ FAIL'}")
print(f"Upload: {'✅ PASS' if upload_ok else '❌ FAIL'}")


🧪 ARDUINO COMPILATION & UPLOAD TEST
🔨 Testing Arduino compilation...
⏰ Started at: 12:28:04
🖥️ Running: arduino-cli compile --fqbn arduino:avr:mega --verbose plasticbottle_rvm.ino

📤 Return code: 0

📤 STDOUT (last 10 lines):
  /home/naufal/.arduino15/packages/arduino/tools/avr-gcc/7.3.0-atmel3.6.1-arduino7/bin/avr-objcopy -O ihex -j .eeprom --set-section-flags=.eeprom=alloc,load --no-change-warnings --change-section-lma .eeprom=0 /home/naufal/.cache/arduino/sketches/B4B9845D603054E03CF2D7C3160D8787/plasticbottle_rvm.ino.elf /home/naufal/.cache/arduino/sketches/B4B9845D603054E03CF2D7C3160D8787/plasticbottle_rvm.ino.eep
  /home/naufal/.arduino15/packages/arduino/tools/avr-gcc/7.3.0-atmel3.6.1-arduino7/bin/avr-objcopy -O ihex -R .eeprom /home/naufal/.cache/arduino/sketches/B4B9845D603054E03CF2D7C3160D8787/plasticbottle_rvm.ino.elf /home/naufal/.cache/arduino/sketches/B4B9845D603054E03CF2D7C3160D8787/plasticbottle_rvm.ino.hex
  /home/naufal/.arduino15/packages/arduino/tools/avr-gcc/7.3.0-

In [4]:
def test_serial_connection():
    """Test koneksi serial ke Arduino"""
    print("\n🔌 Testing serial connection...")
    
    try:
        arduino = serial.Serial(ARDUINO_PORT, 9600, timeout=2)
        time.sleep(2)  # Wait for connection
        
        print(f"✅ Serial connection established on {ARDUINO_PORT}")
        print(f"📋 Port settings: {arduino.baudrate} baud, timeout={arduino.timeout}s")
        
        arduino.close()
        return True
        
    except serial.SerialException as e:
        print(f"❌ Serial connection failed: {e}")
        return False
    except Exception as e:
        print(f"❌ Unexpected error: {e}")
        return False

def test_arduino_communication():
    """Test komunikasi dengan Arduino"""
    print("\n💬 Testing Arduino communication...")
    
    try:
        arduino = serial.Serial(ARDUINO_PORT, 9600, timeout=3)
        time.sleep(2)
        
        # Test 1: Send BOTOL command
        print("\n🧪 Test 1: Sending 'BOTOL' command...")
        arduino.write(b'BOTOL\n')
        
        response_count = 0
        start_time = time.time()
        
        while time.time() - start_time < 5:  # 5 second timeout
            if arduino.in_waiting:
                response = arduino.readline().decode().strip()
                print(f"📥 Response {response_count + 1}: {response}")
                response_count += 1
                
                if response == "SELESAI":
                    print("✅ BOTOL command completed successfully")
                    break
        
        if response_count == 0:
            print("❌ No response from Arduino")
            return False
        
        time.sleep(1)
        
        # Test 2: Send TIDAK command
        print("\n🧪 Test 2: Sending 'TIDAK' command...")
        arduino.write(b'TIDAK\n')
        
        response_count = 0
        start_time = time.time()
        
        while time.time() - start_time < 5:
            if arduino.in_waiting:
                response = arduino.readline().decode().strip()
                print(f"📥 Response {response_count + 1}: {response}")
                response_count += 1
                
                if response == "SELESAI":
                    print("✅ TIDAK command completed successfully")
                    break
        
        arduino.close()
        
        if response_count > 0:
            print("\n✅ Arduino communication test successful!")
            return True
        else:
            print("\n❌ Arduino communication test failed!")
            return False
            
    except Exception as e:
        print(f"❌ Communication test error: {e}")
        return False

def test_sensor_readings():
    """Test pembacaan sensor IR"""
    print("\n📡 Testing IR sensor readings...")
    
    try:
        arduino = serial.Serial(ARDUINO_PORT, 9600, timeout=2)
        time.sleep(2)
        
        # Send command untuk membaca sensor
        arduino.write(b'BOTOL\n')
        
        while True:
            if arduino.in_waiting:
                response = arduino.readline().decode().strip()
                print(f"📥 Arduino: {response}")
                
                if response.startswith("poin:"):
                    points = response.split(":")[1]
                    print(f"🎯 Detected sensors and calculated points: {points}")
                    
                    if points == "0":
                        print("⚠️ No sensors triggered (normal if no bottle present)")
                    else:
                        print(f"✅ Sensors working - Points calculated: {points}")
                        
                elif response == "SELESAI":
                    break
        
        arduino.close()
        print("✅ Sensor test completed")
        return True
        
    except Exception as e:
        print(f"❌ Sensor test error: {e}")
        return False

# Test komunikasi serial
print("\n🧪 ARDUINO COMMUNICATION TEST")
print("=" * 50)

serial_ok = test_serial_connection()

if serial_ok:
    comm_ok = test_arduino_communication()
    sensor_ok = test_sensor_readings()
else:
    print("⚠️ Skipping communication tests due to serial connection failure")
    comm_ok = False
    sensor_ok = False

print("\n" + "=" * 50)
print("📊 COMMUNICATION TEST RESULTS:")
print(f"Serial Connection: {'✅ PASS' if serial_ok else '❌ FAIL'}")
print(f"Arduino Communication: {'✅ PASS' if comm_ok else '❌ FAIL'}")
print(f"Sensor Readings: {'✅ PASS' if sensor_ok else '❌ FAIL'}")


🧪 ARDUINO COMMUNICATION TEST

🔌 Testing serial connection...
✅ Serial connection established on /dev/ttyACM0
📋 Port settings: 9600 baud, timeout=2s

💬 Testing Arduino communication...

🧪 Test 1: Sending 'BOTOL' command...
📥 Response 1: poin:5
📥 Response 2: SELESAI
✅ BOTOL command completed successfully

🧪 Test 2: Sending 'TIDAK' command...
📥 Response 1: SELESAI
✅ TIDAK command completed successfully

✅ Arduino communication test successful!

📡 Testing IR sensor readings...
📥 Arduino: poin:5
🎯 Detected sensors and calculated points: 5
✅ Sensors working - Points calculated: 5
📥 Arduino: SELESAI
✅ Sensor test completed

📊 COMMUNICATION TEST RESULTS:
Serial Connection: ✅ PASS
Arduino Communication: ✅ PASS
Sensor Readings: ✅ PASS


In [5]:
# Final System Validation
print("\n🎯 COMPLETE SYSTEM VALIDATION")
print("=" * 60)

# Collect all test results
test_results = {
    'Arduino CLI': cli_ok,
    'Arduino Core': core_ok,
    'Arduino File': file_ok,
    'Arduino Port': port_ok,
    'Compilation': compile_ok,
    'Upload': upload_ok,
    'Serial Connection': serial_ok,
    'Communication': comm_ok,
    'Sensor Readings': sensor_ok
}

# Display detailed results
print("📊 DETAILED TEST RESULTS:")
for test_name, result in test_results.items():
    status = "✅ PASS" if result else "❌ FAIL"
    print(f"  {test_name:<20}: {status}")

# Calculate overall status
passed_tests = sum(test_results.values())
total_tests = len(test_results)
success_rate = (passed_tests / total_tests) * 100

print(f"\n📈 SUCCESS RATE: {passed_tests}/{total_tests} ({success_rate:.1f}%)")

if success_rate >= 90:
    print("🎉 SYSTEM STATUS: EXCELLENT - Ready for production!")
elif success_rate >= 70:
    print("🟡 SYSTEM STATUS: GOOD - Minor issues need attention")
elif success_rate >= 50:
    print("🟠 SYSTEM STATUS: FAIR - Several issues need fixing")
else:
    print("🔴 SYSTEM STATUS: POOR - Major issues require immediate attention")

# Recommendations
print("\n💡 RECOMMENDATIONS:")

if not cli_ok:
    print("  • Install Arduino CLI")
if not core_ok:
    print("  • Install Arduino AVR core")
if not file_ok:
    print("  • Check Arduino source file")
if not port_ok:
    print("  • Check Arduino USB connection")
if not compile_ok:
    print("  • Fix Arduino code compilation errors")
if not upload_ok:
    print("  • Fix Arduino upload issues")
if not serial_ok:
    print("  • Check serial port permissions")
if not comm_ok:
    print("  • Verify Arduino code is running correctly")
if not sensor_ok:
    print("  • Check IR sensor connections")

if all(test_results.values()):
    print("  🎯 All systems operational! Ready to run main detection program.")

print("\n" + "=" * 60)
print(f"🕐 Diagnostic completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("🔧 Use this information to troubleshoot any issues before running the main program.")


🎯 COMPLETE SYSTEM VALIDATION
📊 DETAILED TEST RESULTS:
  Arduino CLI         : ✅ PASS
  Arduino Core        : ✅ PASS
  Arduino File        : ✅ PASS
  Arduino Port        : ✅ PASS
  Compilation         : ✅ PASS
  Upload              : ✅ PASS
  Serial Connection   : ✅ PASS
  Communication       : ✅ PASS
  Sensor Readings     : ✅ PASS

📈 SUCCESS RATE: 9/9 (100.0%)
🎉 SYSTEM STATUS: EXCELLENT - Ready for production!

💡 RECOMMENDATIONS:
  🎯 All systems operational! Ready to run main detection program.

🕐 Diagnostic completed at: 2025-05-24 12:28:20
🔧 Use this information to troubleshoot any issues before running the main program.


## Troubleshooting Guide

### Common Issues & Solutions:

#### 1. Arduino CLI Not Found
```bash
# Install Arduino CLI
curl -fsSL https://raw.githubusercontent.com/arduino/arduino-cli/master/install.sh | sh

# Add to PATH
echo 'export PATH=$PATH:~/bin' >> ~/.bashrc
source ~/.bashrc
```

#### 2. Arduino Core Missing
```bash
# Install Arduino AVR core
arduino-cli core install arduino:avr

# List available cores
arduino-cli core list
```

#### 3. Permission Denied on Serial Port
```bash
# Add user to dialout group
sudo usermod -a -G dialout $USER

# Set port permissions
sudo chmod 666 /dev/ttyACM0

# Logout and login again
```

#### 4. Wrong Arduino Board
```bash
# List connected boards
arduino-cli board list

# Common board types:
# arduino:avr:uno (Arduino Uno)
# arduino:avr:mega (Arduino Mega)
# arduino:avr:nano (Arduino Nano)
```

#### 5. Upload Errors
- Check USB cable connection
- Try different USB port
- Press reset button on Arduino
- Check if another program is using the port

### Next Steps:
Jika semua test PASS, Anda siap menjalankan program utama untuk deteksi botol plastik!

In [6]:
# Quick Test - Jalankan ini untuk test cepat
print("⚡ QUICK ARDUINO TEST")
print("=" * 30)

try:
    # Test 1: Check file
    if os.path.exists(ARDUINO_FILE):
        print("✅ Arduino file found")
    else:
        print("❌ Arduino file missing")
        
    # Test 2: Check port
    if os.path.exists(ARDUINO_PORT):
        print("✅ Arduino port available")
    else:
        print("❌ Arduino port not found")
        
    # Test 3: Quick serial test
    try:
        ser = serial.Serial(ARDUINO_PORT, 9600, timeout=1)
        ser.close()
        print("✅ Serial connection OK")
    except:
        print("❌ Serial connection failed")
        
    # Test 4: Arduino CLI
    try:
        result = subprocess.run(['arduino-cli', 'version'], 
                              capture_output=True, timeout=5)
        if result.returncode == 0:
            print("✅ Arduino CLI working")
        else:
            print("❌ Arduino CLI error")
    except:
        print("❌ Arduino CLI not found")
        
except Exception as e:
    print(f"❌ Quick test error: {e}")
    
print("\n💡 Run the complete diagnostic above for detailed testing")

⚡ QUICK ARDUINO TEST
✅ Arduino file found
✅ Arduino port available
✅ Serial connection OK
✅ Arduino CLI working

💡 Run the complete diagnostic above for detailed testing


## Manual Test Commands

Jika ingin test manual via terminal:

### 1. Check Arduino CLI
```bash
arduino-cli version
arduino-cli core list
arduino-cli board list
```

### 2. Compile Arduino Code
```bash
arduino-cli compile --fqbn arduino:avr:uno arduino.ino
```

### 3. Upload to Arduino
```bash
arduino-cli compile --upload --fqbn arduino:avr:uno --port /dev/ttyACM0 arduino.ino
```

### 4. Test Serial Communication
```bash
# Install screen if not available
sudo apt install screen

# Connect to Arduino
screen /dev/ttyACM0 9600

# Type: BOTOL
# Expected response: poin:X and SELESAI
# Exit with: Ctrl+A then K
```

### 5. Check Available Ports
```bash
ls /dev/tty*
dmesg | grep tty
```